In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 

In [2]:
!pip install transformers 

In [3]:
!python -m pip install huggingface_hub

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

# Load the model and tokenizer from Hugging Face Hub
model = AutoModelForSequenceClassification.from_pretrained("Ansh007/FineTuneBert01")
tokenizer = AutoTokenizer.from_pretrained("Ansh007/FineTuneBert01")


config.json:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:

# # Define the id2label dictionary
# id2label = {idx: label for idx, label in enumerate(model.config.id2label)}

# def get_answer(text, model, tokenizer, id2label, threshold=0.3):
#     encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
#     encoding = {k: v.to(model.device) for k, v in encoding.items()}
    
#     with torch.no_grad():
#         outputs = model(**encoding)
#     logits = outputs.logits
    
#     sigmoid = torch.nn.Sigmoid()
#     probs = sigmoid(logits.squeeze().cpu())
    
#     predictions = (probs >= threshold).numpy().astype(int)
    
#     predicted_labels = {id2label[idx]: prob.item() for idx, prob in enumerate(probs) if predictions[idx] == 1}
    
#     return predicted_labels



In [6]:
# Define the id2label dictionary from the model configuration
id2label = model.config.id2label

def get_answer(text, model, tokenizer, id2label, threshold=0.4):
    # Tokenize and encode the input text
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    encoding = {k: v.to(model.device) for k, v in encoding.items()}
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    
    # Apply sigmoid to get probabilities
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    
    # Apply threshold to get binary predictions
    predictions = (probs >= threshold).numpy().astype(int)
    
    # Map predictions to labels
    predicted_labels = {id2label[idx]: prob.item() for idx, prob in enumerate(probs) if predictions[idx] == 1}
    
    return predicted_labels

In [7]:
# Example usage
text = ""

predicted_labels = get_answer(text, model, tokenizer, id2label)
print(predicted_labels)


{'regulator': 0.9988659620285034, 'virtual assets': 0.9307739734649658, 'compliance': 0.9736314415931702}
